In [1]:
from pymatgen.io.vasp import Vasprun
from pymatgen.symmetry.bandstructure import HighSymmKpath
from pymatgen.electronic_structure.core import Spin, Orbital
from numpy import savetxt, loadtxt, array, vstack
import os

#JOSTAINS SYYSTA JOSKUS TULEE ERROR JOS VAIN "vasprun.xml"
#This runs off of the vasprunxml

kansio = ""

if not os.path.exists(kansio + "./banddata"):
    os.makedirs(kansio + "./banddata")


vasprun_bands = kansio + "./vasprun.xml"
kpoints_bands = kansio + "./KPOINTS"

run = Vasprun(vasprun_bands, parse_projected_eigen=True)
bands = run.get_band_structure(kpoints_bands,
                           line_mode=True)

if os.path.isfile(kansio + "./efermi.dat"):
    EF = float(loadtxt(kansio + "./efermi.dat"))
    print("Reading EF from banddata/efermi.dat, found EF =  " + str(EF))
else:
    EF = bands.efermi
    print("Reading EF from vasprun.xml, found EF = " + str(EF))

Reading EF from vasprun.xml, found EF = 10.36502442


In [2]:
#Plots up and down spin sepratelys
upvyot=[]
downvyot=[]

for band in bands.bands[Spin.up]:
        vyo = array( band ) - EF
        upvyot.append(vyo)

if len(bands.bands) == 1:

    savetxt("banddata/bandstructure.dat",upvyot)
    savetxt("banddata/bandstructure_up.dat",upvyot)

else:
    for band in bands.bands[Spin.down]:
        vyo = array( band ) - EF
        downvyot.append(vyo)

    savetxt("banddata/bandstructure.dat",upvyot+downvyot)
    savetxt("banddata/bandstructure_up.dat",upvyot)
    savetxt("banddata/bandstructure_down.dat",downvyot)

In [3]:
eig = run.eigenvalues
projeig = run.projected_eigenvalues

nbands = len(projeig[Spin.up][0])
kpoints = run.actual_kpoints
nkpoints = len(kpoints)
weights = run.actual_kpoints_weights
N_atoms = len(projeig[Spin.up][0][0])

for kpts_to_skip in range(len(weights)):
     if weights[kpts_to_skip] == 0: break

print("Found E_fermi       = %f" %EF)
print("Found nkpts_to_skip = %i" %kpts_to_skip)
print("Total # of k-points = %i" %nkpoints)
print("Total # of bands    = %i" %nbands)

#For pbe calc just set kpts_to_skip to 0
#kpoint_indices = range(kpts_to_skip,nkpoints)
kpoint_indices = range(0,nkpoints)


Found E_fermi       = 10.365024
Found nkpts_to_skip = 359
Total # of k-points = 360
Total # of bands    = 24


In [4]:
def filt( outstring, atoms, orbs):
    
        if run.parameters["ISPIN"] == 2:
            spinit = [Spin.up,Spin.down]
            f_up   = open("banddata/" + outstring + "_up.dat", 'w')
            f_down = open("banddata/" + outstring + "_down.dat", 'w')
        else:
            spinit = [Spin.up]
            f_up   = open("banddata/" + outstring + "_up.dat", 'w')
        for spini in spinit:
            for band in range(nbands):
                totalweights = []
                for k in kpoint_indices:
                    totalweight = 0
                    for atom in atoms:
                        for orb in orbs:
                            totalweight += projeig[spini][k][band][atom][orb]
                    totalweights += [totalweight]
                totalweights = array(totalweights)
                if spini == Spin.up:
                        f_up.write("".join([str(tot) + " " for tot in totalweights])+"\n")
                else:
                        f_down.write("".join([str(tot) + " " for tot in totalweights])+"\n")
                #tama ei mystisesti toiminut wtf
                #f_up.write("".join([str(tot*weights_up[band][nkpoints-k-1+kpts_to_skip]) + " " for tot in totalweights])+"\n")

"""
ORBITAL INDICES:
0   s
1   py
2   pz
3   px
4   dxy
5   dyz
6   dz2        
7   dxz
8   dx2-y2
"""
#CHeck numbers and adjust for axes if rotated 45deg

dx2 = [8]               # 45deg SC rotation: xy <--> x2-y2
dxz = [7]
dyz = [5]
dxzyz = dxz + dyz
dz2 = [6]
dxy = [4]               # 45deg SC rotation: xy <--> x2-y2
dt2g = dyz + dxz + dxy
d = [4,5,6,7,8]
p = [1,2,3]
px = [3]
py = [1]
pz = [2]
pxpy = [1,3]
s = [0]

all_orbs=s+p+d

#################################################################

all_atoms = range(N_atoms)

filt("dx2", all_atoms, dx2)
filt("dxy", all_atoms, dxy)
filt("dxz", all_atoms, dxz)
filt("dyz", all_atoms, dyz)
filt("dz2", all_atoms, dz2)



#for c in Cu:
#        filt("Cu%i_dx2"%c, [c], dx2)
#        filt("Cu%i_dz2"%c, [c], dz2)
#        filt("Cu%i_dxzyz"%c, [c], dxzyz)
#        filt("Cu%i_dxz"%c, [c], dxz)
#        filt("Cu%i_dyz"%c, [c], dyz)
#        filt("Cu%i_dxy"%c, [c], dxy)
